<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat</span>
<span><img src="media/inria-25-alpha.png" /></span>
</div>

# plusieurs sortes de merge

In [ ]:
# ce sera toujours notre façon de commencer
[ -f scripts/helpers.sh ] && source scripts/helpers.sh

## reprenons ..

si vous avez bien suivi et exécuté ce qui précède, vous devez avoir un répertoire `my-first-repo`:

* qui contient 6 commits
* et deux branches `master` et `devel`
* la branche `master` pointe sur notre premier merge
* la branche `devel` est un peu en retrait

In [ ]:
# si nécessaire, vous pouvez remettre le repository en l'état
# 
# pour cela mettez "true" au lieu de ""
# et bien sûr évaluer la cellule

reset=""

if [ -n "$reset" ]; then 
    cd $TOP
    bash $SCRIPTS/2-01-my-first-repo.sh
    bash $SCRIPTS/2-02-consistency-repo-fs.sh
    bash $SCRIPTS/2-03-my-first-merge.sh
fi >& /dev/null

## le point..

In [ ]:
# si nécessaire, on se place dans le dépôt git
[ -d my-first-repo ] && cd my-first-repo

pwd

In [ ]:
# vous devez avoir deux branches, 6 commits dont un merge
# et être sur la branche master 
git l --all

## le(s) parent(s) d'un commit

![](../media/commit-parents.png)

* chaque commit a un ou des *parents*
* qui correspondent aux commits  
  sur lequel il est construit

sur cet exemple:

* le parent de B est A
* le parent de E est A
* H a deux parents, D et G
* ...

## commits comparables ou pas

* cette relation définit un **ordre partiel** sur les commits
* selon que deux commits **sont comparables ou non**
  * la création d'un merge va avoir des effets très différents

`master` et `devel` non comparables  
`master` $\nless$ `devel`  
`master` $\ngtr$ `devel`  



![](../media/order-2-not-compare.png) 

ils sont comparables  
`master` > `devel`

![](../media/order-3-compare.png) 

## deux sortes de merge

* si les deux points du merge **sont comparables** :
  * **pas besoin** de créer un commit !
* dans le cas contraire
  * un nouveau commit **est nécessaire**
  
* voyons ces deux cas plus en détail  

## (1) merge avec deux commits comparables

si les deux commits sont comparables

* le plus grand des deux **contient déjà** les changements communs aux deux commits
* d'ailleurs: le plus proche ancêtre commun est le plus petit des deux


## (1) merge avec deux commits comparables

en vert: la branche `current` avant de merger `to_merge`  
en bleu: la position de `current` après le merge

un merge dit *fast-forward*  

![](../media/merge-1-fast-forward.png)

un merge sans aucun effet

![](../media/merge-2-noop.png)

Dans le premier cas :

la branche courante est sur `A`, on fusionne dessus la branche `to_merge`  
mais `to_merge` contient déjà tout le code de `courante`  
ce n'est pas la peine de créer un commit  
on se contente de faire monter `courante` au niveau de `to_merge`
  
Dans le second cas :

cette fois la branche qu'on merge dans `courante` est un ancêtre de `courante`  
donc `courante` contient déjà tout le code de `to_merge`, le merge n'a absolument aucun effet


## (2) merge avec commits incomparables

* dans le cas de deux commits non comparables,  
  il faut **créer un commit** qui incorpore les changements

* c'est fait **automatiquement** par `git merge` 
* toutefois :
  * l'algorithme fonctionne à base de `diff`
  * qui est **orienté lignes**
  * d'où la bien meilleure adéquation sur **le texte**

digression:

* c'est une des raisons qui ont favorisé le format *markdown* 
* on en reparlera

## notion de conflit

* imaginez que vous avez une section de code, disons une ligne
* qui est changée dans la branche A
* et **aussi** changée dans la branche B
* mais de manière différente..

## *auto merge failed*

![](../media/auto-merge-failed.png)

## en cas de conflit

* la fusion automatique  
  **ne peut pas réussir**

* et le commit  **n'est pas créé**

`git merge` va

* fusionner le maximum
  * les cas non conflictuels
  * sont gérés normalement
  * et **mis dans l'index**

* pour le reste
  * il insère des **balises**  
    avec les deux versions

  * **pas dans l'index**
  * vous laisse le soin de choisir

* il vous reste à 
  * résoudre à la main
  * mettre dans l'index  
    les conflits résolus

  * créer le commit vous même

## ex. 1: fast-forward

In [ ]:
git l

* rappel: on avait fusionné `devel` dans `master`
* vérifiez que `master` et `devel` sont comparables
* que se passe-t-il si on merge `master` dans `devel` ?

Pour expliciter le terme employé ici, quand on dit *on a fusionné `devel` dans `master`*, on veut dire plus précisément que, alors que `master` était la branche courante, on a fait `git merge devel`.

In [ ]:
git checkout devel
git merge master

In [ ]:
git l

Réponse: 

* pas de commit créé
* `devel` "rattrape" simplement `master`

Comme on est dans le cas où les deux branches du merge sont comparables, on est dans le cas du *fast-forward* et il n'est pas besoin de créer un commit de fusion; vérifiez la présence du terme *Fast-forward* dans la sortie du `git merge`

## ex. 2 : merge avec conflit - 1ère branche

In [ ]:
# un changement qui
# ne sera pas conflictuel

$SCRIPTS/do no-worries-1

In [ ]:
# celui-ci par contre le sera

$SCRIPTS/do conflict-1

In [ ]:
git diff

On prépare un merge avec conflit, et dans un premier temps on fabrique un commit qui contient deux changements,
l'un ne posera pas de problème alors que l'autre entrera en conflit avec la deuxième branche du merge.

## ex. 2 : merge avec conflit - 1ère branche

In [ ]:
git add factorial.md
git commit -m 'pour conflit dans devel'

In [ ]:
git l --all

## ex. 2 : merge avec conflit - 2ème branche

In [ ]:
# remettons-nous au commit précédent
git checkout master

In [ ]:
# même logique, on fait deux changements

$SCRIPTS/do no-worries-2
$SCRIPTS/do conflict-2

In [ ]:
git diff

In [ ]:
git add factorial.py factorial.md

git commit -m'pour conflit, dans master'

Même chose dans la deuxième branche, on crée un commit qui a deux changments, l'un des deux ne pose pas de problème alors que le second est en conflit avec notre première branche.

## ex. 2 : on a deux branches

In [ ]:
# la situation juste avant le merge
git l --all


## ex. 2 : le merge échoue

on a tout fait pour cela, le merge échoue  

In [ ]:
# on est sur master
git merge devel

Le message d'échec peut paraître relativement sybillin ; en fait les détails de ce qui a réussi ou pas sont donnés à l'utilisateur au travers de l'état dans lequel le dépôt est laissé après le merge.

**à noter** : dans ce cas de figure on pourrait revenir à la situation avant le merge en faisant `git merge --abort`

## ex. 2 : situation après le merge

en cas de conflit :

* les conflits sont annotés dans le fichier concerné  
  les deux versions sont présentes  
  avec des marqueurs qui indiquent la provenance  
  (exemple slide suivant)

* les changements mergés avec succès  
  sont ajoutés à l'index

In [ ]:
# les changements non
# conflictuels sont dans
# l'index
# les conflits se voient
# dans les fichiers 
# concernés

git status

In [ ]:
# voici comment est
# annotée la zone 
# avec conflit

cat factorial.md

Après le merge, les actions du merge qui ont **bien fonctionné** ont été mis **dans l'index**, alors que ce qui est problématique reste dans les changements non indexés.

## ex. 2 : résoudre le conflit

* modifier à la main le⋅s fichier⋅s concerné⋅s
* ajouter ce changement à l'index
* on peut alors committer

## ex. 2 : résoudre le conflit (2)

In [ ]:
# je simule une modification sous éditeur
$SCRIPTS/do resolve-conflict

cat factorial.md

In [ ]:
# maintenant on peut mettre 
# la résolution du conflit dans l'index
git add factorial.md

In [ ]:
# et à présent on peut committer
git commit -m 'conflit résolu'

In [ ]:
git l --all -3

*Rappel :* on utilise ici `git commit -m` pour préciser sur la ligne de commande le message associé à ce commit de merge. On y est contraint car le cours tourne dans un notebook. Sans cette option, `git commit` lance un éditeur de texte dans le terminal pour la saisie du message.

## exercice 1

je vous invite à vous amuser à faire les diffs dans tous les sens :

* calculer le sha-1 du dernier point de fourche ("*mon premier merge*")
* diff avec HEAD (a.k.a. master)
* diff avec devel
* diff entre master et devel

In [ ]:
git diff devel master


## exercice 2

s'entraîner à faire des merge 

1. avec des configurations *claires*, 
   c'est-à-dire où clairement les deux changements sont indépendants les uns des autres
   
1. on a vu un conflit quand les deux branches **modifient** la même ligne  
   que se passe-t-il si les deux branches **insèrent** du code au même endroit dans un fichier

## résumé

* lorsque deux commits sont comparables
  * c'est-à-dire qu'il existe un chemin uniquement descendant de l'un à l'autre
  * alors un merge **ne produit pas de commit**
  * mais peut provoquer un "rattrapage" d'une branche par une autre

* dans le cas contraire
  * un merge **crée un commit** de fusion
  * sauf lorsqu'il y a conflit
  * dans ce cas c'est à vous de résoudre ces conflits à la main
  * et de terminer le travail en créant le commit  

## état

comme d'habitude nous observons notre répo à ce stade

In [ ]:
git l --all